In [1]:
!pip install --upgrade --quiet openai

#### Moderation

In [2]:
import os, openai

with open('gram_token.txt', 'r') as file:
    os.environ['OPENAI_API_KEY'] = file.read().strip()
    
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
from openai import OpenAI

client = OpenAI()

In [7]:
response = client.moderations.create(input = "Why are women bad developers?")
output = response.results[0]
output

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=True, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=0.3030817210674286, harassment_threatening=5.598323014055495e-07, hate=0.41028425097465515, hate_threatening=8.564888389628322e-07, self_harm=6.107258254672843e-09, self_harm_instructions=1.0522284155101147e-09, self_harm_intent=8.864877165670748e-10, sexual=1.0595258572720923e-05, sexual_minors=2.332646090508206e-06, violence=0.0001568168809171766, violence_graphic=1.5803103451617062e-05, self-harm=6.107258254672843e-09, sexual/minors=2.332646090508206e-06, hate/threatening=8.564888389628322e-07, violence/graphic=1.580

In [8]:
output.to_dict()

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'hate': True,
  'hate/threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'sexual': False,
  'sexual/minors': False,
  'violence': False,
  'violence/graphic': False},
 'category_scores': {'harassment': 0.3030817210674286,
  'harassment/threatening': 5.598323014055495e-07,
  'hate': 0.41028425097465515,
  'hate/threatening': 8.564888389628322e-07,
  'self-harm': 6.107258254672843e-09,
  'self-harm/instructions': 1.0522284155101147e-09,
  'self-harm/intent': 8.864877165670748e-10,
  'sexual': 1.0595258572720923e-05,
  'sexual/minors': 2.332646090508206e-06,
  'violence': 0.0001568168809171766,
  'violence/graphic': 1.5803103451617062e-05},
 'flagged': True}

In [10]:
for key, value in output.categories.to_dict().items():
    if value == True :
        print(f"{key} : {value}")

hate : True


In [11]:
output.category_scores.hate

0.41028425097465515

In [15]:
def get_moderation_results(text):
    response = client.moderations.create(input = text)
    output = response.results[0].to_dict()
    flagged = output['flagged']
    rules_contravened = []
    
    for key, value in output["categories"].items():
        if value == True :
            rules_contravened.append(key)
            
    
    print("\n moderation test result")
    print("--------------------------")
    print(f"moderation test text : {text}")
    print(f"moderation test pass ?  : {flagged}")
    print(f"moderation problem categories : {','.join(rules_contravened)}")


In [17]:
get_moderation_results("why do black people commit crimes?")


 moderation test result
--------------------------
moderation test text : why do black people commit crimes?
moderation test pass ?  : True
moderation problem categories : harassment,hate


#### Grounding

In [19]:
!pip install --quiet ragas

In [21]:
from datasets import Dataset
from ragas import evaluate
from ragas. metrics import answer_relevancy, faithfulness, context_recall, answer_correctness, context_precision